In [4]:
import pandas as pd
from pathlib import Path

In [80]:
df=pd.read_csv("../data/processed/clean_data.csv",index_col=0)
df.head()

C:\Users\Rajasvi\Anaconda3\envs\ece143\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,report_id,caers_created_date,date_of_event,product_type,product,product_code,description,patient_age,age_units,sex,medra_preferred_terms,outcomes
0,172934,1/1/2014,12/5/2013,SUSPECT,VALUED NATIONAL PINE NUTS,23,Nuts/Edible Seed,NaN,NaN,NaN,"DYSGEUSIA, HYPERSENSITIVITY",Other Outcome
1,172937,1/1/2014,NaN,SUSPECT,"DAVID'S SUNFLOWER SEEDS, RANCH FLAVORED",23,Nuts/Edible Seed,NaN,NaN,NaN,"PAIN, MUCOSAL ULCERATION, TENDERNESS, BURNING ...","Hospitalization, Patient Visited Healthcare Pr..."
2,172939,1/1/2014,NaN,SUSPECT,"KASHI WHOLE WHEAT BISCUITS, ISLAND VANILLA",5,Cereal Prep/Breakfast Food,NaN,NaN,NaN,"VOMITING, ABDOMINAL PAIN, FEELING OF BODY TEMP...",Other Outcome
3,172940,1/1/2014,NaN,SUSPECT,DANNON DANNON LITE & FIT GREEK YOGURT CHERRY,9,Milk/Butter/Dried Milk Prod,NaN,NaN,NaN,NAUSEA,Other Outcome
4,172941,1/1/2014,12/16/2013,SUSPECT,COPPER RIVER KIPPERED ALASKA SILVER SALMON,16,Fishery/Seafood Prod,NaN,NaN,NaN,LACERATION,Other Outcome


In [65]:
raw_dir = Path("../data/raw/")

aggReports=None

def clean_data(x):
    if isinstance(x,str):
        x=x.strip()
    return x
    

for p in list(raw_dir.glob("*.csv")):
    curr_df = pd.read_csv(p,encoding="unicode_escape")
    
    column_map = { x:x.lower().replace(" ","_") for x in curr_df.columns}
    curr_df=curr_df.rename(columns=column_map)
    curr_df=curr_df.rename(columns={"meddra_preferred_terms":"medra_preferred_terms"})
    curr_df = curr_df.applymap(clean_data)
    
    aggReports = curr_df if aggReports is None else pd.concat([aggReports,curr_df])

len(aggReports)

aggReports.write_csv


131261

In [66]:
aggReports.head()

,report_id,caers_created_date,date_of_event,product_type,product,product_code,description,patient_age,age_units,sex,medra_preferred_terms,outcomes
0,172934,1/1/2014,12/5/2013,SUSPECT,VALUED NATIONAL PINE NUTS,23,Nuts/Edible Seed,NaN,NaN,NaN,"DYSGEUSIA, HYPERSENSITIVITY",Other Outcome
1,172937,1/1/2014,NaN,SUSPECT,"DAVID'S SUNFLOWER SEEDS, RANCH FLAVORED",23,Nuts/Edible Seed,NaN,NaN,NaN,"PAIN, MUCOSAL ULCERATION, TENDERNESS, BURNING ...","Hospitalization, Patient Visited Healthcare Pr..."
2,172939,1/1/2014,NaN,SUSPECT,"KASHI WHOLE WHEAT BISCUITS, ISLAND VANILLA",5,Cereal Prep/Breakfast Food,NaN,NaN,NaN,"VOMITING, ABDOMINAL PAIN, FEELING OF BODY TEMP...",Other Outcome
3,172940,1/1/2014,NaN,SUSPECT,DANNON DANNON LITE & FIT GREEK YOGURT CHERRY,9,Milk/Butter/Dried Milk Prod,NaN,NaN,NaN,NAUSEA,Other Outcome
4,172941,1/1/2014,12/16/2013,SUSPECT,COPPER RIVER KIPPERED ALASKA SILVER SALMON,16,Fishery/Seafood Prod,NaN,NaN,NaN,LACERATION,Other Outcome


In [57]:
# aggReports.report_id = aggReports.report_id.str.strip()
aggReports.dtypes

aggReports[aggReports.report_id=="217550"]

,report_id,caers_created_date,date_of_event,product_type,product,product_code,description,patient_age,age_units,sex,medra_preferred_terms,outcomes


In [75]:
aggReports.groupby(["report_id","product"])
# aggReports[aggReports.report_id=="217550"]

ValueError: must supply a tuple to get_group with multiple grouping keys

In [64]:
(aggReports.isnull().sum()/aggReports.shape[0])*100

report_id                 0.000000
caers_created_date        0.000000
date_of_event            38.439445
product_type              0.000000
product                   0.002286
product_code              0.495958
description               0.495958
patient_age              39.411554
age_units                39.314038
sex                      40.522318
medra_preferred_terms     0.817455
outcomes                  0.000000
dtype: float64

In [57]:
len(aggReports)

131261

In [58]:
aggReports_no_dupes = aggReports.copy()
aggReports_no_dupes.drop_duplicates(subset = 'Report ID',inplace=True)

len(aggReports_no_dupes)

91776

In [60]:
(aggReports_no_dupes.isnull().sum()/aggReports.shape[0])*100

Report ID                  0.000000
CAERS Created Date         0.000000
Date of Event             28.853201
Product Type               0.000000
Product                    0.001524
Product Code               0.474627
Description                0.474627
Patient Age               33.466909
Age Units                 33.391487
Sex                       34.260748
MedDRA Preferred Terms    41.396150
Outcomes                   0.000000
Medra Preferred Terms     29.271452
dtype: float64

In [68]:
aggReports_no_dupes.nunique()

Report ID                 91776
CAERS Created Date         4927
Date of Event              6503
Product Type                  2
Product                   42497
Product Code                 48
Description                  50
Patient Age                 107
Age Units                     5
Sex                           4
MedDRA Preferred Terms    18984
Outcomes                    420
Medra Preferred Terms     23531
time_diff                  4785
time_group                    6
dtype: int64

In [63]:
aggReports_no_dupes["CAERS Created Date"] = pd.to_datetime(aggReports_no_dupes["CAERS Created Date"])
aggReports_no_dupes["Date of Event"] = pd.to_datetime(aggReports_no_dupes["Date of Event"])

aggReports_no_dupes["time_diff"] = (aggReports_no_dupes['CAERS Created Date']-aggReports_no_dupes['Date of Event']).dt.days

# Group the Time difference by different periods
time_list = (0,180,366,1098,1830,3660)
time_cat = ['less than 6 Months','less than 1 year','1 to 3 years','3 to 5 years','5 to 10 Years']
aggReports_no_dupes['time_group'] = pd.cut(aggReports_no_dupes['time_diff'],time_list,labels= time_cat)

aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].astype('str')
aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].replace('nan','Unknown')

In [72]:
aggReports_no_dupes.groupby('time_group')['Report ID'].count()

time_group
1 to 3 years           5304
3 to 5 years           2627
5 to 10 Years          2931
Unknown               40120
less than 1 year       3546
less than 6 Months    37248
Name: Report ID, dtype: int64